In [2]:
import numpy as np
import cv2
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
import random
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array, mode='gaussian', clip=True)


def blur_filter(img_array: ndarray):
    # blur the image
    return cv2.blur(img_array, (3, 3))

def reduce_line_thickness(image: ndarray):
    kernel = np.ones((2,2), np.uint8)
    return cv2.dilate(image, kernel, iterations=1)

def random_stretch(img: ndarray):
    stretch = (random.random() - 0.5)  # -0.5 .. +0.5
    wStretched = max(int(img.shape[1] * (1 + stretch)), 1)  # random width, but at least 1
    img = cv2.resize(img, (wStretched, img.shape[0]))  # stretch horizontally by factor 0.5 .. 1.5
    return img




```
# This is formatted as code
```

# Augmenting IAM dataset

In [5]:
base_path = "drive/My Drive/Handwritten health records"
aug_dir = base_path+"/aug_data/aug_lines/"
folder_dir = base_path+"/data/lines/"

In [6]:
def return_file_names():
    gtTexts = []
    fileNames = []
    with open(base_path+'/data/lines.txt', 'r') as f:
        for line in f:
            # ignore comment line
            if not line or line[0] == '#':
                continue

            lineSplit = line.strip().split(' ')  ## remove the space and split with ' '
            # assert len(lineSplit) >= 9

            # filename: part1-part2-part3 --> part1/part1-part2/part1-part2-part3.png
            fileNameSplit = lineSplit[0].split('-')
            # print(fileNameSplit)
            fileName = fileNameSplit[0] + '/' + fileNameSplit[0] + '-' + fileNameSplit[1] + '/' + lineSplit[0]

            # GT text are columns starting at 9
            print(lineSplit)
            gtText_list = lineSplit[8].split('|')
            gtText = ' '.join(gtText_list)
            # put sample into list
            gtTexts.append(gtText)
            fileNames.append(fileName)
    print(fileNames[0].split('/'))
    return gtTexts, fileNames


def write_text(paths, gtTexts):
    with open(base_path+'/aug_data/line_new.txt', 'w') as f:
        for i in range(len(paths)):
            text = gtTexts[i]
            t = text.split(' ')
            t = '|'.join(t)
            s = paths[i] + " x x x x x x x x " + t
            f.write(s + '\n')
    print("Write Sucessfully")


def create_aug_images():
    gtTexts, fileNames = return_file_names()
    image_paths = []
    image_texts = []
    # dictionary of the transformations functions we defined earlier
    available_transformations = [
        reduce_line_thickness,
        random_noise,
        blur_filter,
        random_stretch
    ]
    # all random numbers generated in a list to choose transformations
    choice = [random.randint(0, len(available_transformations) - 1) for p in range(0, len(gtTexts))]  # between a <= N <= b.
    print(choice)
    for i in range(0, len(gtTexts)):
        # read the image
        img_to_transform = sk.io.imread(folder_dir+fileNames[i]+".png")
        transformed_image = available_transformations[choice[i]](img_to_transform)
        img_name=aug_dir+"iam_aug_"+str(i)+".png"
        sk.io.imsave(img_name, transformed_image)
        image_paths.append("iam_aug_"+str(i)+".png")
        image_texts.append(gtTexts[i])
    write_text(image_paths, image_texts)

In [7]:
create_aug_images()

FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/Handwritten health records/data/lines.txt'

# Augmenting Custom Data

In [ ]:
base_path = "drive/My Drive/Handwritten health records"
aug_dir = base_path+"/custom_data/images/"

In [ ]:
data_path=base_path+"/custom_data/images/"
def get_paths(x):
  return data_path+x

def write_text(paths, gtTexts):
    with open(base_path+'/custom_data/annot1.csv', 'w') as f:
        f.write('images,label\n')
        for i in range(len(paths)):
            text = gtTexts[i]
            s = paths[i] + "," + text
            f.write(s + '\n')
    print("Write Sucessfully")


def create_aug_images():

    data=pd.read_csv(base_path+"/custom_data/annot.csv")
    data['images']=data['images'].apply(lambda x:get_paths(x))

    file_paths=np.array(data['images'])
    gtTexts=np.array(data['label'])

    image_texts =[]
    image_paths=[]

    # dictionary of the transformations functions we defined earlier
    available_transformations = [
        reduce_line_thickness,
        random_noise,
        blur_filter,
        random_stretch
    ]

    for i in range(0, len(gtTexts)):
        # read the image
        img_to_transform = sk.io.imread(file_paths[i])

        for j in range(len(available_transformations)):
          transformed_image = available_transformations[j](img_to_transform)
          img_name=aug_dir+"cust_aug_"+str(i)+"_"+str(j)+".png"
          sk.io.imsave(img_name, transformed_image)
          image_paths.append("cust_aug_"+str(i)+"_"+str(j)+".png")
          gtTexts[i]=gtTexts[i].replace(',',' ')
          image_texts.append(gtTexts[i])
    write_text(image_paths, image_texts)

In [ ]:
create_aug_images()